# Project 3: Office Relocation

## Process: 
1. Choose 3 Spanish cities that are popular in the gaming industry to query for locations.
2. Assign values to all requirements, based on employee value.
3. Write formulae to return the shortest distance for all requirements in the 3 cities.
4. Sum all distance values multiplied by requirement weight to determine the lowest proximity score.
5. Visualize results.

## 1. Determine Locations.

Given this map of all gaming companies globally, I have chosen the most gaming-populous cities to locate the spanish branch of my gaming company: Barcelona, Madrid, and Valencia.

https://gamedevmap.com/index.php?country=Spain&state=&city=&query=&type=

## 2. Assign Value to All Requirements.

Weight value of all employeees: Multiply salary (glassdoor avg) by number of employees in said position.

### Employees:
- 20 Designers
- 5 UI/UX Engineers
- 10 Frontend Developers
- 15 Data Engineers
- 5 Backend Developers
- 20 Account Managers
- 1 Maintenance guy that loves basketball
- 10 Executives
- 1 CEO/President

### Requirements:
- Design companies: Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- School: 30% of the company staff have at least 1 child.
- Startups: Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Starbucks: Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
- Airport: Account managers need to travel a lot.
- Club: Everyone in the company is between 25 and 40, give them some place to go party.
- Vegan restaurant: The CEO is vegan.
- Basketball Stadium: If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- Dog Groomer: The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.

### Scrape Glassdoor for employee salaries:
- add employees to a dataframe.
- url glassdoor {employee name}.
- Web scrape each page for salary, add to df.
- add requirement locations to df.

In [1]:
# import for dataframe usage
import pandas as pd

# import for visual analysis
import folium
from folium import Choropleth, Circle, Marker, Icon, Map, features
from folium.plugins import HeatMap, MarkerCluster

# Import data cleaning and enriching file
from src.functions import*

In [52]:
# outline all information to turn into a dataframe
title = ['Designer', 'UI/UX Engineer', 'Frontend Developer', 'Data Engineer', 'Backend Developer', 'Account Manager', 'Maintenance', 'Executive', 'CEO']
salary_k = [30, 33, 33, 44, 39, 25, 20, 60, 80] # Checked on glassdoor.
amt = [20, 5, 10, 15, 5, 20, 1, 10, 1]
total_salary_score = []
location_req = ['design_companies', 'design_companies', 'startups', 'startups', 'startups', 'airport', 'stadium', 'starbucks', 'vegan_eat']
shared_reqs = ['school', 'club', 'groomer', '-','-','-','-','-','-']

# add salary score to dataframe-to-be
for i in range(len(title)): 
    total_salary_score.append(salary_k[i]*amt[i])

# Dataframe structure: employee_title, employee_salary, employees_amt, total_score, location_req
to_df = {'title':title, 'salary (k)':salary_k, 'amt':amt, 'total salary score':total_salary_score, 'location req':location_req, 'shared reqs':shared_reqs}
df = pd.DataFrame.from_dict(to_df)

# visualize dataframe and decide on rankings
df

,title,salary (k),amt,total salary score,location req,shared reqs
0,Designer,30,20,600,design_companies,school
1,UI/UX Engineer,33,5,165,design_companies,club
2,Frontend Developer,33,10,330,startups,groomer
3,Data Engineer,44,15,660,startups,-
4,Backend Developer,39,5,195,startups,-
5,Account Manager,25,20,500,airport,-
6,Maintenance,20,1,20,stadium,-
7,Executive,60,10,600,starbucks,-
8,CEO,80,1,80,vegan_eat,-


## Employee Value Analysis
### Call me an American, but...
The most sensible proposition is to surround your business with a professional network. Having more businesses nearby to propogate more business: Startups first, and then design companies, as recommended by the salary scores of developers and designers. Schools are vital to 30% of the company and are deal-breakers for some fo the older, high-level executives. Then comes Starbucks for all the executives, which is not hard to find and useful by score. Clubs and other nightlife allow younger employees to propogate a company culture that will attract the interest of nearby companies. 

An airport is important for the account managers (a vital asset to the company) but everyone expects a 45 minute taxi ride from the airport for any city, so as long as the airport is within a certain radius the location is good. The stadium would rank lower, but several employees have chimed in about their interest in attending events with the maintenance man.

With a company of 87 people, the CEO is phasing out her office time to network with investors, explore new businesses, and spend time with her family. Her needs to have a vegan restaurant nearby are low, as we confirmed in our last 1:1. A dog groomer ranked lowest because everyone loves Fido but it does not affect anyone in a significant manner, and dogs in Spain are notoriously ungroomed anyway.

The total score of all employees is 2,550, so group scores are calculated in fractions: 30% for schools (765), 20% for clubs (510), and 1% for the dog (25). The stadium was also boosted by 10% of the whole (250) because other employees also enjoy this location.

### Final results:
1. Startups (1185)
2. Design Companies (765)
3. Schools (765)
4. Starbucks (600)
5. Clubs (510)
6. Airport (500)
7. Stadium (275)
8. Vegan Eats (80)
9. Dog Groomer (25)

## 3. Query the Shortest Distances for All Employee Requirements.

### Process:
- Pass a list of potential office locations.
- Calculate the shortest ditance from each requirement for each potential office.

Dataframe structure: office_location, design_companies, school, startups, starbucks, airport, club, vegan_eat, stadium, groomer

In [3]:
mad = find_offices('Madrid')
bcn = find_offices('Barcelona')
val = find_offices('Valencia')

mad.head(5)

,name,lat,lon
0,Fon,40.530724,-3.632687
1,Buongiorno,40.448797,-3.669764
2,Swotti,40.416741,-3.703250
3,AsesorSeguros,40.497716,-3.647842
4,eRepublik,40.430940,-3.695290


In [4]:
bcn.head(5)

,name,lat,lon
0,eyeOS,41.385486,2.170287
1,]project-open[,41.383182,2.163697
2,Starfish Community Group,41.387917,2.169919
3,Trovit,41.388769,2.129949
4,BMAT,41.406335,2.192023


In [5]:
val # we don't need to call 'head' because there are only 5 companies here

,name,lat,lon
0,e-contratos,39.469301,-0.377068
1,Modo Marketing,39.640730,-0.421944
2,Efebege,39.450211,-0.392465
3,DOMODESK,39.461268,-0.398346
4,H2scan,34.442143,-118.598765


### Query Requirements: Startups

Query Foursquare Query for the nearest startups.

In [6]:
# this was almost accomplished in the 'find_offices' function with a mongodb query but the mongodb database is incomplete.

# query Foursquare for the closest venue
val_nearest_startup = get_venue('startup',val)
val_nearest_startup

,name,startup distance,startup lat,startup lon
0,e-contratos,129,39.469993,-0.375853
1,Modo Marketing,10444,39.552132,-0.462790
2,Efebege,1132,39.459232,-0.386196
3,DOMODESK,865,39.465750,-0.388911
4,H2scan,1102,34.438503,-118.587461


In [8]:
# Do the same for MAD and BCN
bcn_nearest_startup = get_venue('startup',bcn)
mad_nearest_startup = get_venue('startup',mad)

### Location Requirement: Designers

Query MongoDB for the nearest design companies. 2nd attempt (after limited results): query Foursquare.

Add map graphic below, keep functions to visualize in a spearate function doc.

In [12]:
# query Foursquare for the closest venue
val_nearest_designer = get_venue('design',val)
bcn_nearest_designer = get_venue('design',bcn)
mad_nearest_designer = get_venue('design',mad)

### Query Requirements: Starbucks

Query Foursquare for the nearest Starbucks.

In [13]:
# query Foursquare for the closest venue
val_nearest_starbucks = get_venue('starbucks',val)
bcn_nearest_starbucks = get_venue('starbucks',bcn)
mad_nearest_starbucks = get_venue('starbucks',mad)

### Query Requirements: Airport

Query Foursquare for the nearest airport.

In [14]:
# query Foursquare for the closest venue
val_nearest_airport = get_venue('airport',val)
bcn_nearest_airport = get_venue('airport',bcn)
mad_nearest_airport = get_venue('airport',mad)

### Query Requirements: Schools

Query Foursquare for the nearest schools.

In [15]:
# query Foursquare for the closest venue
val_nearest_school = get_venue('school',val)
bcn_nearest_school = get_venue('school',bcn)
mad_nearest_school = get_venue('school',mad)

### Query Requirements: Club
Note: this is a communal decision.
Query Foursquare for the nearest club.

In [16]:
# query Foursquare for the closest venue
val_nearest_club = get_venue('club',val)
bcn_nearest_club = get_venue('club',bcn)
mad_nearest_club = get_venue('club',mad)

### Query Requirements: Vegan Restaurant

Query Foursquare for the nearest vegan restaurant.

In [17]:
# query Foursquare for the closest venue
val_nearest_vegan = get_venue('vegan',val)
bcn_nearest_vegan = get_venue('vegan',bcn)
mad_nearest_vegan = get_venue('vegan',mad)

### Query Requirements: Stadium

Query Foursquare for the nearest stadium.

In [18]:
# query Foursquare for the closest venue
val_nearest_stadium = get_venue('stadium',val)
bcn_nearest_stadium = get_venue('stadium',bcn)
mad_nearest_stadium = get_venue('stadium',mad)

### Query Requirements: Dog Groomer
Notice: this is a communal decision.
Query Foursquare for the nearest dog groomer.

In [19]:
# query Foursquare for the closest venue
val_nearest_groomer = get_venue('pet services',val)
bcn_nearest_groomer = get_venue('pet services',bcn)
mad_nearest_groomer = get_venue('pet services',mad)

### Merge All Dataframes

In [20]:
# merge madrid dataframes for easy access
df_list = [mad, mad_nearest_startup, mad_nearest_designer, mad_nearest_starbucks, mad_nearest_airport, 
           mad_nearest_school, mad_nearest_club, mad_nearest_vegan, mad_nearest_stadium, mad_nearest_groomer]
import functools as ft
mad_complete = ft.reduce(lambda left, right: pd.merge(left, right, on='name'), df_list)

# merge barcelona dataframes for easy access
df_list = [bcn, bcn_nearest_startup, bcn_nearest_designer, bcn_nearest_starbucks, bcn_nearest_airport, 
           bcn_nearest_school, bcn_nearest_club, bcn_nearest_vegan, bcn_nearest_stadium, bcn_nearest_groomer]
import functools as ft
bcn_complete = ft.reduce(lambda left, right: pd.merge(left, right, on='name'), df_list)

# merge valencia dataframes for easy access
df_list = [val, val_nearest_startup, val_nearest_designer, val_nearest_starbucks, val_nearest_airport, 
           val_nearest_school, val_nearest_club, val_nearest_vegan, val_nearest_stadium, val_nearest_groomer]
import functools as ft
val_complete = ft.reduce(lambda left, right: pd.merge(left, right, on='name'), df_list)

# save dataframes to csvs
mad_complete.to_csv("output/madrid_office_proximity.csv")
bcn_complete.to_csv("output/barcelona_office_proximity.csv")
val_complete.to_csv("output/valencia_office_proximity.csv")

val_complete

,name,lat,lon,startup distance,startup lat,startup lon,design distance,design lat,design lon,starbucks distance,...,club lon,vegan distance,vegan lat,vegan lon,stadium distance,stadium lat,stadium lon,pet services distance,pet services lat,pet services lon
0,e-contratos,39.469301,-0.377068,129,39.469993,-0.375853,290,39.469913,-0.373776,144,...,-0.375490,47,39.469071,-0.377547,311,39.466551,-0.376036,1142,39.459688,-0.383472
1,Modo Marketing,39.640730,-0.421944,10444,39.552132,-0.462790,12235,39.595920,-0.552432,17226,...,-0.539931,10064,39.594372,-0.320787,6973,39.585700,-0.461105,11722,39.579130,-0.533096
2,Efebege,39.450211,-0.392465,1132,39.459232,-0.386196,409,39.448897,-0.388003,2559,...,-0.383420,1813,39.460272,-0.376026,718,39.452388,-0.400345,1363,39.459688,-0.383472
3,DOMODESK,39.461268,-0.398346,865,39.465750,-0.388911,866,39.465528,-0.389309,2031,...,-0.393719,1180,39.469470,-0.389599,1001,39.452388,-0.400345,1305,39.459688,-0.383472
4,H2scan,34.442143,-118.598765,1102,34.438503,-118.587461,2126,34.434036,-118.578185,1266,...,-118.581779,2290,34.442697,-118.573834,2380,34.452466,-118.621615,3299,34.436411,-118.563348


## 4. Determine Proximity Score.

Weighted value of all requirements: Multiply weighted ranks by distance to each location to find the best possible new branch office in each city.

1. Startups (1185)
2. Design Companies (765)
3. Schools (765)
4. Starbucks (600)
5. Clubs (510)
6. Airport (500)
7. Stadium (275)
8. Vegan Eats (80)
9. Dog Groomer (25)

In [61]:
bcn_complete

,name,lat,lon,startup distance,startup lat,startup lon,design distance,design lat,design lon,starbucks distance,...,club lon,vegan distance,vegan lat,vegan lon,stadium distance,stadium lat,stadium lon,pet services distance,pet services lat,pet services lon
0,eyeOS,41.385486,2.170287,58,41.385957,2.170607,133,41.384199,2.180161,205,...,2.170654,178,41.384408,2.168819,1021,41.382617,2.158648,778,41.386723,2.161102
1,]project-open[,41.383182,2.163697,233,41.384878,2.165442,346,41.404413,2.164952,252,...,2.162859,58,41.382539,2.163428,425,41.382617,2.158648,449,41.386723,2.161102
2,Starfish Community Group,41.387917,2.169919,118,41.389033,2.169356,180,41.387770,2.172331,168,...,2.171603,163,41.387534,2.167993,1060,41.383810,2.158438,746,41.386723,2.161102
3,Trovit,41.388769,2.129949,47,41.388623,2.130295,176,41.389258,2.128659,56,...,2.128576,107,41.388255,2.131036,962,41.396978,2.126265,265,41.386995,2.130976
4,BMAT,41.406335,2.192023,22,41.406400,2.192177,1094,41.398486,2.199841,146,...,2.190953,503,41.402411,2.195277,495,41.402592,2.188790,1223,41.402951,2.205989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,quimica_visual,41.372303,2.156075,469,41.375546,2.152471,542,41.376642,2.155620,624,...,2.158721,371,41.375261,2.153641,564,41.367356,2.155735,211,41.374286,2.156309
1058,U-Play Studios,41.406335,2.192023,22,41.406400,2.192177,1094,41.398486,2.199841,145,...,2.190953,503,41.402411,2.195277,495,41.402592,2.188790,1223,41.402951,2.205989
1059,Iris Experience,41.392701,2.156538,265,41.394236,2.154104,259,41.394560,2.154658,294,...,2.156972,155,41.391875,2.155207,1000,41.383810,2.158438,765,41.386723,2.161102
1060,iContainers,41.376304,2.175271,281,41.378220,2.177475,258,41.378045,2.177417,616,...,2.175070,265,41.378006,2.172954,1553,41.382617,2.158648,1608,41.374286,2.156309


In [36]:
# TEST FUNCTION
def closest_office(city):
    lowest_score = sys.maxsize
    lowest_coords = []
    for i in city.index.values.tolist():
        sum_of_weighted_distances = city.at[i,'startup distance']*1185 + city.at[i,'design distance']*765 + city.at[i,'starbucks distance']*600 
        + city.at[i,'school distance']*765 + city.at[i,'club distance']*510 + city.at[i,'stadium distance']*275 + city.at[i,'vegan distance']*80 + city.at[i,'pet services distance']*25
        if city.at[i,'airport distance'] > 20000: # check if airport is too far away (longer than 45 min taxi/20km)
            sum_of_weighted_distances =0
        if sum_of_weighted_distances < lowest_score:
            lowest_score = sum_of_weighted_distances
            while len(lowest_coords) > 0:
                lowest_coords.pop()
            lowest_coords.append([city.at[i,'lat'], city.at[i,'lon']])
    return lowest_coords

In [53]:
# loop through a list of all office locations and compare to their respective distances from each venue
madrid_office = closest_office(mad_complete)
barcelona_office = closest_office(bcn_complete)
valencia_office = closest_office(val_complete)

# display test values
valencia_office

[[39.469301, -0.377068]]

In [47]:
madrid_office

[[40.4161835, -3.6988729]]

In [39]:
barcelona_office

[[41.388769, 2.1299487]]

## 5. Visualize Results.
plot the top office choice from each city on a folium map.

## Madrid

In [59]:
# draw a map of the city
venue_location = [madrid_office[0][0],madrid_office[0][1]]
map_3 = folium.Map(venue_location, zoom_start=17)

# set all icons
venue_icon = Icon(color="orange",
           opacity=0.1,
           prefix="fa",
           icon="building",
           icon_color="black")
venue_marker = folium.Marker(venue_location, tooltip="Venue", icon=venue_icon)
venue_marker.add_to(map_3)

design_location = [mad_complete['design lat'][1073], mad_complete['design lon'][1073]]
coordinates = [venue_location, design_location]
distance = mad_complete.at[0,"design distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='pink',weight=10)
map_3.add_child(my_PolyLine)
design_icon = Icon(color="pink",
           opacity=0.1,
           prefix="fa",
           icon="paint-brush",
           icon_color="black")
design_marker = folium.Marker(design_location, tooltip="Design Company", icon=design_icon)
design_marker.add_to(map_3)

school_location = [mad_complete['school lat'][1073], mad_complete['school lon'][1073]]
coordinates = [venue_location, school_location]
distance = mad_complete.at[0,"school distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='purple',weight=10)
map_3.add_child(my_PolyLine)
school_icon = Icon(color="purple",
           opacity=0.1,
           prefix="fa",
           icon="graduation-cap",
           icon_color="black")
school_marker = folium.Marker(school_location, tooltip="School", icon=school_icon)
school_marker.add_to(map_3)

tech_location = [mad_complete['startup lat'][1073], mad_complete['startup lon'][1073]]
coordinates = [venue_location, tech_location]
distance = mad_complete.at[0,"startup distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='blue',weight=10)
map_3.add_child(my_PolyLine)
tech_icon = Icon(color="blue",
           opacity=0.1,
           prefix="fa",
           icon="desktop",
           icon_color="black")
tech_marker = folium.Marker(tech_location, tooltip="Startup", icon=tech_icon)
tech_marker.add_to(map_3)

cafe_location = [mad_complete['starbucks lat'][1073], mad_complete['starbucks lon'][1073]]
coordinates = [venue_location, cafe_location]
distance = mad_complete.at[0,"starbucks distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkgreen',weight=10)
map_3.add_child(my_PolyLine)
cafe_icon = Icon(color="darkgreen",
           opacity=0.1,
           prefix="fa",
           icon="coffee",
           icon_color="black")
cafe_marker = folium.Marker(cafe_location, tooltip="Starbucks", icon=cafe_icon)
cafe_marker.add_to(map_3)

airport_location = [mad_complete['airport lat'][1073], mad_complete['airport lon'][1073]]
coordinates = [venue_location, airport_location]
distance = mad_complete.at[0,"airport distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: 8976m",color='gray',weight=10)
map_3.add_child(my_PolyLine)
airport_icon = Icon(color="gray",
           opacity=0.1,
           prefix="fa",
           icon="plane",
           icon_color="black")
airport_marker = folium.Marker(airport_location, tooltip="Airport", icon=airport_icon)
airport_marker.add_to(map_3)

party_location = [mad_complete['club lat'][1073], mad_complete['club lon'][1073]]
coordinates = [venue_location, party_location]
distance = mad_complete.at[0,"club distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkred',weight=10)
map_3.add_child(my_PolyLine)
party_icon = Icon(color="darkred",
           opacity=0.1,
           prefix="fa",
           icon="beer",
           icon_color="black")
party_marker = folium.Marker(party_location, tooltip="Club", icon=party_icon)
party_marker.add_to(map_3)

vegan_location = [mad_complete['vegan lat'][1073], mad_complete['vegan lon'][1073]]
coordinates = [venue_location, vegan_location]
distance = mad_complete.at[0,"vegan distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='cadetblue',weight=10)
map_3.add_child(my_PolyLine)
vegan_icon = Icon(color="cadetblue",
           opacity=0.1,
           prefix="fa",
           icon="cutlery",
           icon_color="black")
vegan_marker = folium.Marker(vegan_location, tooltip="Vegan Restaurant", icon=vegan_icon)
vegan_marker.add_to(map_3)

basketball_location = [mad_complete['stadium lat'][1073], mad_complete['stadium lon'][1073]]
coordinates = [venue_location, basketball_location]
distance = mad_complete.at[0,"stadium distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='lightgreen',weight=10)
map_3.add_child(my_PolyLine)
basketball_icon = Icon(color="lightgreen",
           opacity=0.1,
           prefix="fa",
           icon="support",
           icon_color="black")
basketball_marker = folium.Marker(basketball_location, tooltip="Stadium", icon=basketball_icon)
basketball_marker.add_to(map_3)

dog_location = [mad_complete['pet services lat'][1073], mad_complete['pet services lon'][1073]]
coordinates = [venue_location, dog_location]
distance = mad_complete.at[0,"pet services distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='red',weight=10)
map_3.add_child(my_PolyLine)
dog_icon = Icon(color="red",
           opacity=0.1,
           prefix="fa",
           icon="paw",
           icon_color="black")
dog_marker = folium.Marker(dog_location, tooltip="Dog Groomer", icon=dog_icon)
dog_marker.add_to(map_3)

# visualize the map!
map_3

## Barcelona

In [63]:
# draw a map of the city
venue_location = [barcelona_office[0][0], barcelona_office[0][1]]
map_2 = folium.Map(venue_location, zoom_start=17)

# set all icons
venue_icon = Icon(color="orange",
           opacity=0.1,
           prefix="fa",
           icon="building",
           icon_color="black")
venue_marker = folium.Marker(venue_location, tooltip="Venue", icon=venue_icon)
venue_marker.add_to(map_2)

design_location = [bcn_complete['design lat'][3], bcn_complete['design lon'][3]]
coordinates = [venue_location, design_location]
distance = bcn_complete.at[0,"design distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='pink',weight=10)
map_2.add_child(my_PolyLine)
design_icon = Icon(color="pink",
           opacity=0.1,
           prefix="fa",
           icon="paint-brush",
           icon_color="black")
design_marker = folium.Marker(design_location, tooltip="Design Company", icon=design_icon)
design_marker.add_to(map_2)

school_location = [bcn_complete['school lat'][3], bcn_complete['school lon'][3]]
coordinates = [venue_location, school_location]
distance = bcn_complete.at[0,"school distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='purple',weight=10)
map_2.add_child(my_PolyLine)
school_icon = Icon(color="purple",
           opacity=0.1,
           prefix="fa",
           icon="graduation-cap",
           icon_color="black")
school_marker = folium.Marker(school_location, tooltip="School", icon=school_icon)
school_marker.add_to(map_2)

tech_location = [bcn_complete['startup lat'][3], bcn_complete['startup lon'][3]]
coordinates = [venue_location, tech_location]
distance = bcn_complete.at[0,"startup distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='blue',weight=10)
map_2.add_child(my_PolyLine)
tech_icon = Icon(color="blue",
           opacity=0.1,
           prefix="fa",
           icon="desktop",
           icon_color="black")
tech_marker = folium.Marker(tech_location, tooltip="Startup", icon=tech_icon)
tech_marker.add_to(map_2)

cafe_location = [bcn_complete['starbucks lat'][3], bcn_complete['starbucks lon'][3]]
coordinates = [venue_location, cafe_location]
distance = bcn_complete.at[0,"starbucks distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkgreen',weight=10)
map_2.add_child(my_PolyLine)
cafe_icon = Icon(color="darkgreen",
           opacity=0.1,
           prefix="fa",
           icon="coffee",
           icon_color="black")
cafe_marker = folium.Marker(cafe_location, tooltip="Starbucks", icon=cafe_icon)
cafe_marker.add_to(map_2)

airport_location = [41.2974, 2.0833] # search returned the 'airport hotel'
coordinates = [venue_location, airport_location]
distance = bcn_complete.at[0,"airport distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: 8976m",color='gray',weight=10)
map_2.add_child(my_PolyLine)
airport_icon = Icon(color="gray",
           opacity=0.1,
           prefix="fa",
           icon="plane",
           icon_color="black")
airport_marker = folium.Marker(airport_location, tooltip="Airport", icon=airport_icon)
airport_marker.add_to(map_2)

party_location = [bcn_complete['club lat'][3], bcn_complete['club lon'][3]]
coordinates = [venue_location, party_location]
distance = bcn_complete.at[0,"club distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkred',weight=10)
map_2.add_child(my_PolyLine)
party_icon = Icon(color="darkred",
           opacity=0.1,
           prefix="fa",
           icon="beer",
           icon_color="black")
party_marker = folium.Marker(party_location, tooltip="Club", icon=party_icon)
party_marker.add_to(map_2)

vegan_location = [bcn_complete['vegan lat'][3], bcn_complete['vegan lon'][3]]
coordinates = [venue_location, vegan_location]
distance = bcn_complete.at[0,"vegan distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='cadetblue',weight=10)
map_2.add_child(my_PolyLine)
vegan_icon = Icon(color="cadetblue",
           opacity=0.1,
           prefix="fa",
           icon="cutlery",
           icon_color="black")
vegan_marker = folium.Marker(vegan_location, tooltip="Vegan Restaurant", icon=vegan_icon)
vegan_marker.add_to(map_2)

basketball_location = [bcn_complete['stadium lat'][3], bcn_complete['stadium lon'][3]]
coordinates = [venue_location, basketball_location]
distance = bcn_complete.at[0,"stadium distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='lightgreen',weight=10)
map_2.add_child(my_PolyLine)
basketball_icon = Icon(color="lightgreen",
           opacity=0.1,
           prefix="fa",
           icon="support",
           icon_color="black")
basketball_marker = folium.Marker(basketball_location, tooltip="Stadium", icon=basketball_icon)
basketball_marker.add_to(map_2)

dog_location = [bcn_complete['pet services lat'][3], bcn_complete['pet services lon'][3]]
coordinates = [venue_location, dog_location]
distance = bcn_complete.at[0,"pet services distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='red',weight=10)
map_2.add_child(my_PolyLine)
dog_icon = Icon(color="red",
           opacity=0.1,
           prefix="fa",
           icon="paw",
           icon_color="black")
dog_marker = folium.Marker(dog_location, tooltip="Dog Groomer", icon=dog_icon)
dog_marker.add_to(map_2)

# visualize the map!
map_2

## Valencia

In [55]:
# draw a map of the city
venue_location = [valencia_office[0][0], valencia_office[0][1]]
map_1 = folium.Map(venue_location, zoom_start=17)

# set all icons
venue_icon = Icon(color="orange",
           opacity=0.1,
           prefix="fa",
           icon="building",
           icon_color="black")
venue_marker = folium.Marker(venue_location, tooltip="Venue", icon=venue_icon)
venue_marker.add_to(map_1)

design_location = [val_complete['design lat'][0], val_complete['design lon'][0]]
coordinates = [venue_location, design_location]
distance = val_complete.at[0,"design distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='pink',weight=10)
map_1.add_child(my_PolyLine)
design_icon = Icon(color="pink",
           opacity=0.1,
           prefix="fa",
           icon="paint-brush",
           icon_color="black")
design_marker = folium.Marker(design_location, tooltip="Design Company", icon=design_icon)
design_marker.add_to(map_1)

school_location = [val_complete['school lat'][0], val_complete['school lon'][0]]
coordinates = [venue_location, school_location]
distance = val_complete.at[0,"school distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='purple',weight=10)
map_1.add_child(my_PolyLine)
school_icon = Icon(color="purple",
           opacity=0.1,
           prefix="fa",
           icon="graduation-cap",
           icon_color="black")
school_marker = folium.Marker(school_location, tooltip="School", icon=school_icon)
school_marker.add_to(map_1)

tech_location = [val_complete['startup lat'][0], val_complete['startup lon'][0]]
coordinates = [venue_location, tech_location]
distance = val_complete.at[0,"startup distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='blue',weight=10)
map_1.add_child(my_PolyLine)
tech_icon = Icon(color="blue",
           opacity=0.1,
           prefix="fa",
           icon="desktop",
           icon_color="black")
tech_marker = folium.Marker(tech_location, tooltip="Startup", icon=tech_icon)
tech_marker.add_to(map_1)

cafe_location = [val_complete['starbucks lat'][0], val_complete['starbucks lon'][0]]
coordinates = [venue_location, cafe_location]
distance = val_complete.at[0,"starbucks distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkgreen',weight=10)
map_1.add_child(my_PolyLine)
cafe_icon = Icon(color="darkgreen",
           opacity=0.1,
           prefix="fa",
           icon="coffee",
           icon_color="black")
cafe_marker = folium.Marker(cafe_location, tooltip="Starbucks", icon=cafe_icon)
cafe_marker.add_to(map_1)

airport_location = [val_complete['airport lat'][0], val_complete['airport lon'][0]]
coordinates = [venue_location, airport_location]
distance = val_complete.at[0,"airport distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: 8976m",color='gray',weight=10)
map_1.add_child(my_PolyLine)
airport_icon = Icon(color="gray",
           opacity=0.1,
           prefix="fa",
           icon="plane",
           icon_color="black")
airport_marker = folium.Marker(airport_location, tooltip="Airport", icon=airport_icon)
airport_marker.add_to(map_1)

party_location = [val_complete['club lat'][0], val_complete['club lon'][0]]
coordinates = [venue_location, party_location]
distance = val_complete.at[0,"club distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='darkred',weight=10)
map_1.add_child(my_PolyLine)
party_icon = Icon(color="darkred",
           opacity=0.1,
           prefix="fa",
           icon="beer",
           icon_color="black")
party_marker = folium.Marker(party_location, tooltip="Club", icon=party_icon)
party_marker.add_to(map_1)

vegan_location = [val_complete['vegan lat'][0], val_complete['vegan lon'][0]]
coordinates = [venue_location, vegan_location]
distance = val_complete.at[0,"vegan distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='cadetblue',weight=10)
map_1.add_child(my_PolyLine)
vegan_icon = Icon(color="cadetblue",
           opacity=0.1,
           prefix="fa",
           icon="cutlery",
           icon_color="black")
vegan_marker = folium.Marker(vegan_location, tooltip="Vegan Restaurant", icon=vegan_icon)
vegan_marker.add_to(map_1)

basketball_location = [val_complete['stadium lat'][0], val_complete['stadium lon'][0]]
coordinates = [venue_location, basketball_location]
distance = val_complete.at[0,"stadium distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='lightgreen',weight=10)
map_1.add_child(my_PolyLine)
basketball_icon = Icon(color="lightgreen",
           opacity=0.1,
           prefix="fa",
           icon="support",
           icon_color="black")
basketball_marker = folium.Marker(basketball_location, tooltip="Stadium", icon=basketball_icon)
basketball_marker.add_to(map_1)

dog_location = [val_complete['pet services lat'][0], val_complete['pet services lon'][0]]
coordinates = [venue_location, dog_location]
distance = val_complete.at[0,"pet services distance"]
my_PolyLine=folium.PolyLine(locations=coordinates,tooltip=f"Distance: {distance}m",color='red',weight=10)
map_1.add_child(my_PolyLine)
dog_icon = Icon(color="red",
           opacity=0.1,
           prefix="fa",
           icon="paw",
           icon_color="black")
dog_marker = folium.Marker(dog_location, tooltip="Dog Groomer", icon=dog_icon)
dog_marker.add_to(map_1)

# visualize the map!
map_1